In [73]:
!python --version

Python 3.8.10


IMPORT LIBRARIES

In [74]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

import xgboost as xgb
import pickle

STANDARDIZATION

In [75]:
X_train = pd.read_parquet('../processed_data/X_train.parquet')
X_val = pd.read_parquet('../processed_data/X_val.parquet')

y_train = np.loadtxt('../processed_data/y_train.txt').astype(int)
y_val = np.loadtxt('../processed_data/y_val.txt').astype(int)

In [77]:
# Seperation of columns into numeric and categorical columns
num_cols = np.array(X_train.select_dtypes(include= ['int64','float64']).columns).tolist()
cat_cols = np.array(X_train.select_dtypes(include= ['category','object']).columns).tolist()

In [78]:
print(cat_cols)
print(num_cols)
print(set(cat_cols + num_cols) - set(X_train.columns))  # Should be empty

['AGE_GROUP', 'YEARS_EMPLOYED_GROUP', 'PHONE_CHANGE_GROUP']
['REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'EXT_SOURCE_3', 'EXT_SOURCE_2', 'EXT_SOURCE_1', 'FLOORSMAX_AVG']
set()


In [79]:
train_dicts = X_train[cat_cols + num_cols].to_dict(orient='records')
val_dicts = X_val[cat_cols + num_cols].to_dict(orient='records')

In [80]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

EXPERIMENT TRACKING

In [81]:
import mlflow

mlflow.set_tracking_uri('sqlite:///../cred_risk_sqlite_mlflow.db')
mlflow.set_experiment('credit_default_risk_experiment_tracking')

<Experiment: artifact_location='/Users/mac/Projects/MLops_credit_default_risk_prediction/02-experiment-tracking/mlruns/1', creation_time=1754811864406, experiment_id='1', last_update_time=1754811864406, lifecycle_stage='active', name='credit_default_risk_experiment_tracking', tags={}>

In [82]:

with mlflow.start_run():
    mlflow.set_tag("engineer", "adeakinwe")
    mlflow.set_tag("model", "Logistic Regression")

    mlflow.log_param("train_data_path", "../processed_data/X_train.parquet")
    mlflow.log_param("val_data_path", "../processed_data/X_val.parquet")

    lr = LogisticRegression(class_weight='balanced')
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    y_proba = lr.predict_proba(X_val)[:, 1]

    accuracy = round(accuracy_score(y_val, y_pred), 3)
    auc = round(roc_auc_score(y_val, y_proba), 3)

    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("auc", auc)

In [83]:
with mlflow.start_run():
    mlflow.set_tag("engineer", "adeakinwe")
    mlflow.set_tag("model", "XGBoost")

    mlflow.log_param("train_data_path", "../processed_data/X_train.parquet")
    mlflow.log_param("val_data_path", "../processed_data/X_val.parquet")

    # Prepare DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)

    # Compute scale_pos_weight if needed
    class_counts = pd.Series(y_train).value_counts()
    scale_pos_weight = class_counts[0] / class_counts[1]

    # XGBoost params
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "scale_pos_weight": scale_pos_weight,
    }

    mlflow.log_param("scale_pos_weight", scale_pos_weight)

    # Train with early stopping
    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=100,
        evals=[(dtrain, "train"), (dval, "eval")],
        early_stopping_rounds=50,
        verbose_eval=False
    )

    # Predict and evaluate
    y_pred_proba = model.predict(dval)
    y_pred = (y_pred_proba > 0.5).astype(int)

    accuracy = round(accuracy_score(y_val, y_pred), 3)
    auc = round(roc_auc_score(y_val, y_pred_proba), 3)

    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("auc", auc)

HYPER-PARAMETER TUNING

In [84]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
import mlflow.xgboost

In [85]:
# Search space for hyperopt
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
}

# Objective function for hyperopt
def objective(params):
    # Calculate imbalance ratio
    neg, pos = np.bincount(y_train)
    scale_pos_weight = neg / pos

    # Add required static params
    params['objective'] = 'binary:logistic'
    params['seed'] = 42
    params['eval_metric'] = 'auc'
    params['scale_pos_weight'] = scale_pos_weight

    run_name = f"xgb-md{params['max_depth']}-lr{params['learning_rate']:.3f}"

    with mlflow.start_run(nested=True, run_name=run_name):
        mlflow.set_tag("model", "XGBoost")
        mlflow.set_tag("engineer", "adeakinwe")

        mlflow.log_param("train_data", "../processed_data/X_train.parquet")
        mlflow.log_param("val_data", "../processed_data/X_val.parquet")

        mlflow.log_params({k: round(v, 5) if isinstance(v, float) else v for k, v in params.items()})

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)

        model = xgb.train(
            params,
            dtrain,
            num_boost_round=200,
            evals=[(dval, 'eval')],
            early_stopping_rounds=50,
            verbose_eval=10
        )

        y_pred_proba = model.predict(dval)
        y_pred = (y_pred_proba > 0.5).astype(int)

        accuracy = round(accuracy_score(y_val, y_pred), 3)
        auc = round(roc_auc_score(y_val, y_pred_proba), 3)

        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("auc", auc)

        mlflow.xgboost.log_model(model, artifact_path="models")

        return {'loss': -auc, 'status': STATUS_OK}

# Run outer MLflow parent run
with mlflow.start_run(run_name="xgboost-hyperopt"):
    trials = Trials()
    best_result = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=50,
        trials=trials
    )

    # Log best hyperparameters found
    mlflow.log_params({f"best_{k}": v for k, v in best_result.items()})

[0]	eval-auc:0.61075                                  
[10]	eval-auc:0.66212                                 
[20]	eval-auc:0.66643                                 
[30]	eval-auc:0.66567                                 
[40]	eval-auc:0.66182                                 
[50]	eval-auc:0.65924                                 
[60]	eval-auc:0.65760                                 
[67]	eval-auc:0.65682                                 
  0%|          | 0/50 [00:15<?, ?trial/s, best loss=?]

2025/08/10 09:02:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



[0]	eval-auc:0.60377                                                
[10]	eval-auc:0.64307                                               
[20]	eval-auc:0.65915                                               
[30]	eval-auc:0.66392                                               
[40]	eval-auc:0.66588                                               
[50]	eval-auc:0.66664                                               
[60]	eval-auc:0.66559                                               
[70]	eval-auc:0.66545                                               
[80]	eval-auc:0.66469                                               
[90]	eval-auc:0.66397                                               
[100]	eval-auc:0.66226                                              
[101]	eval-auc:0.66225                                              
  2%|▏         | 1/50 [00:45<18:57, 23.22s/trial, best loss: -0.657]

2025/08/10 09:02:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



[0]	eval-auc:0.61617                                                
[10]	eval-auc:0.68345                                               
[20]	eval-auc:0.68966                                               
[30]	eval-auc:0.68560                                               
[40]	eval-auc:0.67814                                               
[50]	eval-auc:0.67476                                               
[60]	eval-auc:0.67421                                               
[66]	eval-auc:0.67189                                               
  4%|▍         | 2/50 [00:59<21:17, 26.61s/trial, best loss: -0.662]

...

[0]	eval-auc:0.61387                                                 
[10]	eval-auc:0.62516                                                
[20]	eval-auc:0.62298                                                
[30]	eval-auc:0.62132                                                
[40]	eval-auc:0.62349                                                
[50]	eval-auc:0.62330                                                
[58]	eval-auc:0.62225                                                
 92%|█████████▏| 46/50 [22:10<02:24, 36.24s/trial, best loss: -0.734]

2025/08/10 09:24:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



[0]	eval-auc:0.60167                                                 
[10]	eval-auc:0.64483                                                
[20]	eval-auc:0.65690                                                
[30]	eval-auc:0.65999                                                
[40]	eval-auc:0.66015                                                
[50]	eval-auc:0.66076                                                
[60]	eval-auc:0.66061                                                
[70]	eval-auc:0.65894                                                
[80]	eval-auc:0.65749                                                
[90]	eval-auc:0.65856                                                
[100]	eval-auc:0.65852                                               
[101]	eval-auc:0.65863                                               
 94%|█████████▍| 47/50 [23:07<01:31, 30.56s/trial, best loss: -0.734]

2025/08/10 09:25:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



[0]	eval-auc:0.64125                                                 
[10]	eval-auc:0.71102                                                
[20]	eval-auc:0.71701                                                
[30]	eval-auc:0.71690                                                
[40]	eval-auc:0.71330                                                
[50]	eval-auc:0.71118                                                
[60]	eval-auc:0.70889                                                
[70]	eval-auc:0.70651                                                
[75]	eval-auc:0.70482                                                
 96%|█████████▌| 48/50 [23:41<01:22, 41.26s/trial, best loss: -0.734]

train and save best model

In [ ]:
#best params from hyper parameter tuning
best_params = {
    'max_depth': 4,
    'learning_rate': 0.0566,
    'reg_alpha': 0.04744,
    'reg_lambda': 0.04301,
    'min_child_weight': 1.42993,
    'subsample': 0.65727,
    'colsample_bytree': 0.60704,
    'objective': 'binary:logistic',
    'seed': 42,
    'eval_metric': 'auc',
    'scale_pos_weight': 11.38747
}

with mlflow.start_run(run_name="xgboost-final-auc"):

    mlflow.set_tag("model", "XGBoost")
    mlflow.set_tag("engineer", "adeakinwe")
    mlflow.log_param("train_data_path", "../processed_data/X_train.parquet")
    mlflow.log_param("val_data_path", "../processed_data/X_val.parquet")

    mlflow.log_params(best_params)

    # Prepare DMatrix
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)

    # Train model
    model = xgb.train(
        best_params,
        dtrain,
        num_boost_round=200,
        evals=[(dval, 'eval')],
        early_stopping_rounds=50,
        verbose_eval=10
    )

    # Evaluate
    y_proba = model.predict(dval)
    auc = round(roc_auc_score(y_val, y_proba), 3)
    mlflow.log_metric("auc", auc)

    # Log model (native)
    mlflow.xgboost.log_model(model, artifact_path="models/xgboost_model")

    # Dump model + vectorizer together
    model_bundle = {
        "model": model,
        "vectorizer": dv  # dict_vectorizer
    }

    bundle_path = "../models/xgb_credit_pred.bin"
    with open(bundle_path, "wb") as f_out:
        pickle.dump(model_bundle, f_out)

    mlflow.log_artifact(bundle_path)

load model with mlflow run id and predict

In [ ]:
logged_model = 'runs:/2c2f5792316545ed84ddf88b09b072a9/models/xgboost_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

In [ ]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

In [ ]:
y_pred = xgboost_model.predict(dval)

In [ ]:
y_pred[:10]